In [1]:
from sentence_transformers import SentenceTransformer, util

/Users/eliastakala/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
model = SentenceTransformer('all-mpnet-base-v2') #max input length 512 output dimension 384

/Users/eliastakala/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
import numpy as np
import pandas as pd

df = pd.read_csv('translated_course_data_V2.csv')


In [5]:
df['embedding'] = df["course_description_en"].apply(lambda x: model.encode(x.replace("\n", " ")))

In [ ]:
import seaborn as sns
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

df["embedding"] = df["embedding"].apply(lambda x: np.array(x))

# Keep the top 3 faculties
X = df.embedding
faculty = df["course_code"].str.split("-", expand=True)[0]
top_faculties = faculty.value_counts().index[:3]
mask = faculty.isin(top_faculties)
X_masked = np.stack(X[mask].to_numpy())

reduced = TSNE(
    n_components=2,
    perplexity=40,
    n_iter=5000,
).fit_transform(X_masked)

# Faculty labels
_, faculty_labels = np.unique(faculty[mask].values, return_inverse=True)


# Plot
fig, ax = plt.subplots(figsize=(5, 5))
scatter = ax.scatter(
    reduced[:, 0],
    reduced[:, 1],
    c=faculty_labels,
)

# Legend
legend1 = ax.legend(
    scatter.legend_elements()[0],
    top_faculties,
    title="Faculty",
)

sns.despine()
ax.grid(alpha=0.2)
ax.set_axisbelow(True)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("BERT embeddings in 2D", fontweight="bold")
ax.set_xlabel("1st component")
ax.set_ylabel("2nd component")

fig.tight_layout()
plt.show()